In [0]:
from pyspark.sql.functions import sum
df = spark.table("segwise2024.default.google_play_dataset_by_tapivedotcom")

[Row(appId='com.cyou.cma.clauncher.theme.v542279ccb70366ba6adf1de2', developer='Best theme workshop', developerId='7398289236941441420', developerWebsite='http://www.c-launcher.com/', free=1, genre='Art & Design', genreId='ART_AND_DESIGN', inAppProductPrice='None', minInstalls=1000000, offersIAP=0, originalPrice='None', price=0.0, ratings=30650, len screenshots=4, adSupported=1, containsAds=1, reviews=152, releasedDayYear='Sep 27, 2014', sale=0, score=4.75, summary='Cute Bear Theme for Launcher with pink hearts wallpapers and kawaii icon pack!', title='Cute Bear love  honey with Pin', updated=1555921116.0, histogram1=591, histogram2=0, histogram3=1815, histogram4=1477, histogram5=26681, releasedDay=27, releasedYear=2014, releasedMonth='Sep', dateUpdated=datetime.datetime(2019, 4, 22, 8, 18, 36), minprice='None', maxprice=0.0, ParseReleasedDayYear=datetime.datetime(2014, 9, 27, 0, 0)),
 Row(appId='com.cyou.cma.clauncher.theme.v546307d9199ddff33502cbdb', developer='Best Android Themes Wo

In [0]:
df.na.drop()
df.columns

['appId',
 'developer',
 'developerId',
 'developerWebsite',
 'free',
 'genre',
 'genreId',
 'inAppProductPrice',
 'minInstalls',
 'offersIAP',
 'originalPrice',
 'price',
 'ratings',
 'len screenshots',
 'adSupported',
 'containsAds',
 'reviews',
 'releasedDayYear',
 'sale',
 'score',
 'summary',
 'title',
 'updated',
 'histogram1',
 'histogram2',
 'histogram3',
 'histogram4',
 'histogram5',
 'releasedDay',
 'releasedYear',
 'releasedMonth',
 'dateUpdated',
 'minprice',
 'maxprice',
 'ParseReleasedDayYear']

In [0]:
# Bucketing the dataset by genre
genre_cnt = df.select('genre').distinct().count()
if genre_cnt < 100:
  bucket_cnt = genre_cnt
elif genre_cnt< 1000:
  bucket_cnt = genre_cnt
else:
  bucket_cnt = 100
df.write.bucketBy(bucket_cnt,'genre').parquet('bucketed_data')
df = spark.read.parquet('bucketed_data')

DataFrame[appId: string, developer: string, developerId: string, developerWebsite: string, free: bigint, genre: string, genreId: string, inAppProductPrice: string, minInstalls: bigint, offersIAP: bigint, originalPrice: string, price: double, ratings: bigint, len screenshots: bigint, adSupported: bigint, containsAds: bigint, reviews: bigint, releasedDayYear: string, sale: bigint, score: double, summary: string, title: string, updated: double, histogram1: bigint, histogram2: bigint, histogram3: bigint, histogram4: bigint, histogram5: bigint, releasedDay: bigint, releasedYear: bigint, releasedMonth: string, dateUpdated: timestamp, minprice: string, maxprice: double, ParseReleasedDayYear: timestamp]

In [0]:
#filtering out paid apps, selecting specific columns and casting minInstalls to float 
df_paid = df.filter(df.free==0).select('genre','price','minInstalls')
df_paid = df_paid.withColumn('minInstalls',df.minInstalls.cast('float'))

In [0]:
#Top 10 Genres for which consumers are willing to pay for
df_paid.groupBy('genre').agg(sum('minInstalls').alias('min_Installs')).orderBy('min_Installs', ascending=False).na.drop().limit(10).show()

+---------------+------------+
|          genre|min_Installs|
+---------------+------------+
|         Action| 7.2340651E7|
|         Arcade| 3.5410546E7|
|          Tools| 3.0844775E7|
|      Adventure| 2.9810442E7|
|         Puzzle| 2.9726573E7|
|Personalization| 2.6544565E7|
|   Role Playing| 2.2983466E7|
|       Strategy| 2.2966601E7|
|     Simulation| 2.2037837E7|
|  Music & Audio| 1.6843644E7|
+---------------+------------+



In [0]:
#Most popular genres based on the user reviews
df_reviews = df.filter(df.score.isNotNull())
df_reviews.groupBy('genre').agg(sum('reviews').alias('total_reviews'), sum('minInstalls').alias('Num_of_Installs')).orderBy('total_reviews','Num_of_Installs', ascending=False).limit(10).show()


+-------------+-------------+---------------+
|        genre|total_reviews|Num_of_Installs|
+-------------+-------------+---------------+
|       Casual|     18351468|    22216203017|
|       Action|     18033132|    23746658428|
|       Social|     17067320|    18568734632|
|Communication|     16707387|    63150705051|
|        Tools|     16059030|   101609727010|
|       Arcade|     14119614|    16637463350|
|       Puzzle|     12877550|    14506465011|
|   Simulation|     12686311|    15057295126|
|Entertainment|     11570292|    25508821400|
|Music & Audio|     10320033|    16104426948|
+-------------+-------------+---------------+



In [0]:
df.groupBy('genre', 'price').count().orderBy('genre', 'price').show()

+-------------------+-----+-----+
|              genre|price|count|
+-------------------+-----+-----+
|               NULL| NULL|   55|
|           가격비교| NULL|    1|
|             +Falbo| NULL|    1|
|                  0| NULL|   10|
|                 85| NULL|    1|
|8841647080022600382|  0.0|    1|
|             Action|  0.0|40752|
|             Action| 0.05|    1|
|             Action|  0.1|    1|
|             Action| 0.25|    2|
|             Action| 0.26|    2|
|             Action|  0.4|    1|
|             Action| 0.41|    1|
|             Action| 0.45|    6|
|             Action| 0.48|    4|
|             Action| 0.49|   58|
|             Action|  0.6|    1|
|             Action| 0.69|    9|
|             Action| 0.71|    1|
|             Action| 0.76|    1|
+-------------------+-----+-----+
only showing top 20 rows



In [0]:
df.select('inAppProductPrice','sale','offersIAP','originalPrice','price').show()

+-----------------+----+---------+-------------+-----+
|inAppProductPrice|sale|offersIAP|originalPrice|price|
+-----------------+----+---------+-------------+-----+
|             None|   0|        0|         None|  0.0|
|             None|   0|        0|         None|  0.0|
|             None|   0|        0|         None|  0.0|
|             None|   0|        0|         None|  0.0|
|             None|   0|        0|         None| 2.61|
|             None|   0|        0|         None|  9.9|
|             None|   0|        0|         None|  0.0|
|             None|   0|        0|         None|  0.0|
|             None|   0|        0|         None|  0.0|
|   $0.99 per item|   0|        1|         None|  0.0|
|             None|   0|        0|         None|  0.0|
|             None|   0|        0|         None|  0.0|
|             None|   0|        0|         None|  0.0|
|             None|   0|        0|         None|  0.0|
|             None|   0|        0|         None|  0.0|
|         